In [ ]:
from uqer import Client
from uqer import DataAPI as api
from QuantLib import *

In [ ]:
_ = Client(token='f1b9bea1d0b4e489c5ab9b69c3e2326a1bee6057af858067dbd1546453f428b2')

In [ ]:
maturity_date = Date(1, 6, 2018)
start_date = Date(1, 1, 2018)

spot = 1.

risk_free = 0.03
dividend_rate = 0.0
volatility = 0.40
day_count = Actual365Fixed()
calendar = China(China.SSE)

## Market Data
-------------------

In [ ]:
calculation_date = start_date
Settings.instance().evaluationDate = calculation_date

spot_handle = RelinkableQuoteHandle(SimpleQuote(spot))
vol_handle =  RelinkableQuoteHandle(SimpleQuote(volatility))

flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free, day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, vol_handle, day_count))

bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)

## Analytic Digital Option
-----------------------

### 1. 参数
-------------

In [ ]:
strike = 1.1 * spot
cash_pay = 0.05 * spot

### 2. 计算
--------------

In [ ]:
payoff = CashOrNothingPayoff(Option.Call, strike, cash_pay)
exercise = EuropeanExercise(maturity_date)
option = EuropeanOption(payoff, exercise)

In [ ]:
%%time
engine = AnalyticEuropeanEngine(bsm_process)
option.setPricingEngine(engine)

bsm_price = option.NPV()
print("BSM european digital theoreticl price is {0:.2f}%".format(bsm_price * 100.))

## Analytic Spread Option
--------------------------

### 1. 参数
------------------

In [ ]:
strike1 = 1.1 * spot
strike2 = 1.2 * spot

leverage = 0.9

### 2. 计算
-------------

In [ ]:
payoff = GapPayoff(Option.Call, strike1, strike2)
exercise = EuropeanExercise(maturity_date)
option = EuropeanOption(payoff, exercise)

In [ ]:
%%time
engine = AnalyticEuropeanEngine(bsm_process)
option.setPricingEngine(engine)

bsm_price = option.NPV() * leverage
print("BSM european spread theoreticl price is {0:.2f}%".format(bsm_price * 100.))